<h2>This bot trades 5EMA in Nifty by buying put options</h2>
<hr>

In [1]:
import numpy as np
import pandas as pd
import yfinance as yf
import talib as ta
import time
from pushover import Pushover
import logging
from kiteconnect import KiteConnect

In [2]:
po = Pushover("an3he6x85bwsh97t87goxwu462yru6")
po.user("uuev1ffw9cepf5iquv3wu8hk29xo3m")

In [3]:
logging.basicConfig(level=logging.DEBUG)
kite = KiteConnect(api_key="9qnpf0zzf6pa6cx1")
data = kite.generate_session("kDgBF1C8ZBYhiwbXsKiQjRpS5tUxLAXU", api_secret="dt58xuvzpq1mxkmvvak1zu2uftz6cl63")
kite.set_access_token(data["access_token"])

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "POST /session/token HTTP/1.1" 200 None


<hr>
<h3>Auto target exit not Implemented yet, Watch market after order has been executed</h3>
<hr>

In [3]:
def myround(x):
    return 50 * round(x/50)

In [4]:
def Watch(target,sl,tsym,entry,n,d,en):
    while(True):
        data = yf.download(tickers='^NSEI', period='2d', interval='5m')
        data['5EMA'] = ta.EMA(data['Close'],5)
        print(data.tail())

        # if(data['Low'].values[-2]-data['5EMA'].values[-2]>0.4):
        #     msg = po.msg("Watch the market")
        #     msg.set("title", "Possible Reversal")
        #     po.send(msg)

        if((data['Close'].values[-1]<target) or (data['Low'].values[-1]<target)):
            msg = po.msg("Watch the market")
            msg.set("title", "Target Hit")
            po.send(msg)
            sl = en - 5
            en = target
            n += 1
            target = entry - (n*d)
            Watch(target,sl,tsym,entry,n,d,en)

        
        if(data['Close'].values[-1]>sl):
            try:
                s_order_id = kite.place_order(tradingsymbol=tsym,
                                exchange=kite.EXCHANGE_NSE,
                                transaction_type=kite.TRANSACTION_TYPE_SELL,
                                quantity=50,
                                variety=kite.VARIETY_REGULAR,
                                order_type=kite.ORDER_TYPE_MARKET,
                                product=kite.PRODUCT_MIS)
                logging.info("Order placed. ID :{}".format(s_order_id))
                msg = po.msg("Stop loss hit Check P&L ID :{}".format(s_order_id))
                msg.set("title", "Stop Loss hit")
                po.send(msg)
            except Exception as e:
                msg = po.msg("Failed: "+str(e))
                logging.info("Buy back manually "+str(e))
                msg.set("title", "Stop loss failed")
                po.send(msg)
            msg = po.msg("Square off position and Watch the market")
            msg.set("title", "Stop loss Hit")
            po.send(msg)
            return 
        
        time.sleep(5)

In [5]:
def Enter():
    while(True):
        data = yf.download(tickers='^NSEI', period='2d', interval='5m')
        data['5EMA'] = ta.EMA(data['Close'],5)
        print(data.tail())
        if(data['Low'].values[-1]<data['5EMA'].values[-2] or data['Close'].values[-1]<data['5EMA'].values[-2]):
            sl = max(data['High'].values[-3],data['High'].values[-2],data['High'].values[-4])
            tsym = "NIFTY23302"+str(myround(data['Close'][-1]))+"PE"
            try:
                b_order_id = kite.place_order(tradingsymbol=tsym,
                                exchange=kite.EXCHANGE_NSE,
                                transaction_type=kite.TRANSACTION_TYPE_BUY,
                                quantity=50,
                                variety=kite.VARIETY_REGULAR,
                                order_type=kite.ORDER_TYPE_MARKET,
                                product=kite.PRODUCT_MIS)
                logging.info("Order placed. ID :{}".format(b_order_id))
                msg = po.msg("Buy order placed ID :{}".format(b_order_id))
                msg.set("title", "PE bought")
                po.send(msg)
            except Exception as e:
                msg = po.msg("Failed: "+str(e))
                logging.info("Order placement failed: "+str(e))
                msg.set("title", "Buy Order Failed")
                po.send(msg)
            entry = data['Close'].values[-1]
            target = entry - (sl - entry)
            sle = ((sl-entry)*0.5)*50
            msg = po.msg("Expected max loss :{}".format(sle))
            msg.set("title", "Sl set")
            po.send(msg)
            n = 1
            d = sl - entry
            en = entry
            Watch(target,sl,tsym,entry,n,d,en)
            return 
        time.sleep(5)

In [6]:
def Trade():
    while(True):
        data = yf.download(tickers='^NSEI', period='2d', interval='5m')
        data['5EMA'] = ta.EMA(data['Close'],5)
        print(data.tail())
        if(data['Low'].values[-2]>data['5EMA'].values[-2]):
            msg = po.msg("Watch market about to place order")
            msg.set("title", "Indicator hit")
            po.send(msg)
            Enter()
            msg = po.msg("Rerun the bot")
            msg.set("title", "Trade cycle complete")
            po.send(msg)
            return 
        time.sleep(300)

In [8]:
Trade()

[*********************100%***********************]  1 of 1 completed
                             Open          High           Low         Close  \
Datetime                                                                      
2023-03-02 14:45:00  17323.650391  17332.250000  17322.449219  17328.000000   
2023-03-02 14:50:00  17328.500000  17333.550781  17314.349609  17327.050781   
2023-03-02 14:55:00  17326.800781  17329.099609  17324.000000  17327.900391   
2023-03-02 15:00:00  17328.800781  17336.199219  17319.650391  17327.949219   
2023-03-02 15:05:24  17330.849609  17330.849609  17330.849609  17330.849609   

                        Adj Close  Volume          5EMA  
Datetime                                                 
2023-03-02 14:45:00  17328.000000       0  17329.196983  
2023-03-02 14:50:00  17327.050781       0  17328.481582  
2023-03-02 14:55:00  17327.900391       0  17328.287852  
2023-03-02 15:00:00  17327.949219       0  17328.174974  
2023-03-02 15:05:24  17330.84

Exception: inputs are all NaN